### 1.优化
加入正则项

In [1]:
import tensorflow as tf

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# 获取一层神经网络边上的权重，并将这个权重的L2正则化损失加入名称为'losses'的集合中
def get_weight(shape, lambda1):
    #生成一个变量
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    #add_to_collection 函数将这个新生成变量的L2正则化损失项加入集合
    #这个函数第一个元素是，集合的名字，第二个参数是要加入这个集合的内容
    tf.add_to_collection(
    'losses',tf.contrib.layers.l2_regularizer(lambda1)(var))
    return var
    


In [5]:
x = tf.placeholder(tf.float32,[None,2])
y_ = tf.placeholder(tf.float32,[None,1])

batch_size = 8
# 定义了每一层网络中节点的个数
layer_dimension = [2,10,10,10,1]
# 神经网络的层数
n_layers = len(layer_dimension)

# 这个变量维护前向传播时最深的节点，开始的时候是输入层
cur_layer = x
# 当前节点的个数
in_dimension = layer_dimension[0]


In [6]:
# 通过一个循环生成5层全连接神经网络
for i in range(1,n_layers):
    # layer_dimension[i]为下一层节点的个数
    out_dimension = layer_dimension[i]
    # 生成当前层中权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant([0.1],shape=[out_dimension]))
    # 使用ReLU激活函数
    cur_layer = tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    # 进入下一层之前将下一层的节点个数更新为当前节点个数
    in_dimension = layer_dimension[i]

In [8]:
# 在定义神经网络前向传播的同时已经将所有的L2正则损失项加入了图上的集合
# 这里只需要计算刻画模型在训练数据上表现的损失函数
mse_loss = tf.reduce_mean(tf.square(y_ -cur_layer))
# 将均方误差损失函数加入损失集合
tf.add_to_collection('losses',mse_loss)

### 2. 滑动平均模型

In [9]:
import tensorflow as tf

# 定义一个变量用于计算滑动平均，这个变量的初始值为0 。注意这里手动指定了变量的类型为tf.float32，因为所有需要计算滑动平均的变量必须是实数型
v1 = tf.Variable(0,dtype=tf.float32)
# 这里step变量模拟神经网络中迭代的轮数，可以用于动态控制衰减率
step = tf.Variable(0,trainable=False)

# 定义一个滑动平均的类，初始化时给定了衰减率(0.99)和控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99,step)

# 定义一个更新变量滑动平均的操作，这里需要给定一个列表，每次执行这个操作时，这个列表中的变量都会被更新
maintain_averages_op = ema.apply([v1])

In [11]:
with tf.Session() as sess:
    # 初始化所有变量。
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 通过ema.average(v1)获取滑动平均之后变量的取值。在初始化之后变量v1的值和v1的滑动平均都为0
    print(sess.run([v1,ema.average(v1)]))
    
    # 更新变量v1的值到5
    sess.run(tf.assign(v1,5))
    # 更新v1的滑动平均值。衰减率为 min{0.99,(1+step)/(10+step)=0.1}=0.1
    # 所以v1的滑动平均值会被更新为0.1X0+0.9X5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    # 更新step的值为10000
    sess.run(tf.assign(step,10000))
    #更新v1的值为10
    sess.run(tf.assign(v1,10))
    # 更新v1的滑动平均值。
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    # 再次更新滑动平均值
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
